In [29]:
from pymongo import MongoClient
from riotwatcher import RiotWatcher

In [30]:
DATABASE_NAME = "test"
QUEUE_NAME = "summoner_queue"
URL = "192.168.5.100"

In [15]:
key = raw_input("Please enter your Riot key: ")
target_num = input("Please enter number of games to scrape: ")

watcher = RiotWatcher(key)

KeyboardInterrupt: 

In [31]:
def get_summoner_queue():
    client = MongoClient('mongodb://'+URL)
    
    return client[DATABASE_NAME][QUEUE_NAME]

In [38]:
test = get_summoner_queue()
print test
print type(test)
print test.name
for x in test.find():
    print x

Collection(Database(MongoClient('192.168.5.100', 27017), u'test'), u'summoner_queue')
<class 'pymongo.collection.Collection'>
summoner_queue
{u'_id': ObjectId('556a57ecb751dff5a6808e8f'), u'id': 5012.0}
{u'_id': ObjectId('556a5812b751dff5a6808e90'), u'id': 234.0}
{u'_id': ObjectId('556a584db751dff5a6808e91'), u'id': 8299.0}
{u'_id': ObjectId('556a584db751dff5a6808e92'), u'id': 4959.0}
{u'_id': ObjectId('556a5878b751dff5a6808e93'), u'id': 219.0}
{u'_id': ObjectId('556a5878b751dff5a6808e94'), u'id': 5342.0, u'name': u'Starris'}


In [35]:
def get_next_in_queue(queue):
    return queue.command("findandmodify", "summonerQueue", query = {"queue": "summonerId"}, sort = {"_id": pymongo.ASCENDING}, remove=True)